In [1]:

import datasets

import pandas as pd
import operator

In [2]:
#load original dataset
train_dataset = datasets.load_dataset(path='nlpaueb/finer-139', split='train',streaming = True)
test_dataset = datasets.load_dataset(path='nlpaueb/finer-139', split='test',streaming = True)
validation_dataset = datasets.load_dataset(path='nlpaueb/finer-139', split='validation',streaming = True)

In [3]:
# enumerate dataset, each value is a dict data type, each value has to keys which are tokens and ner_tags
# i extract the tokens and ner_tags from dataset and put them in two lists.
def get_fulldata(dataset):
    tokens = []
    ner_tags = []
    for idx,value in enumerate(dataset):
        tokens.append(value['tokens'])
        ner_tags.append(value['ner_tags'])
    return tokens,ner_tags

In [4]:
train_tokens,train_ner_tags = get_fulldata(train_dataset)
test_tokens,test_ner_tags = get_fulldata(test_dataset)
validation_tokens,validation_ner_tags = get_fulldata(validation_dataset)

In [5]:
def find_label_occurence(ner_tags):
    label_occurence = {}
    label_list = [x for x in range(1,280)]
    for i in label_list:
        for index,value in enumerate(ner_tags):
            for j in value:
                if (j==i):
                     label_occurence[i] = label_occurence.get(i, 0.0) + 1.0
            
    label_occurence = dict(sorted(label_occurence.items(), key=operator.itemgetter(0),reverse=False))           
    return label_occurence 

In [ ]:
label_occurence = find_label_occurence(train_ner_tags)

In [ ]:
def find_number_ratio(label_occurence,tokens,ner_tags):
    number_occurence = {}
    number_ratio = {}
    label_list = [x for x in range(1,280)]
    #if the token is a number, go to find the corrosponding label in the ner_tags list, and plus 1 for the label
    for index,value in enumerate(tokens):
        for idx, token in enumerate(value):
            if re.fullmatch(r"(\d+[\d,.]*)|([,.]\d+)", token):
                label = ner_tags[index][idx]
                number_occurence[label] = number_occurence.get(label, 0.0) + 1.0
    #delete number = 0 key in the dict
    del number_occurence[0]
    #use the  total label occurence divide the number_occurence          
    for key in number_occurence.keys():
        number_ratio[key]=str(round(number_occurence[key]/label_occurence[key]*100,3))+'%'
    #sort by key
    number_ratio = dict(sorted(number_ratio.items(), key=operator.itemgetter(0),reverse=False)) 
    #convert the dict to dataframe
    number = {'label':[],'ratio':[]}
    
    for key in number_ratio.keys():
        number['label'].append(key)
        number['ratio'].append(number_ratio[key])
        
    df = pd.DataFrame.from_dict(number)
    
    return number_ratio,df

In [ ]:
number_ratio,df= find_number_ratio(label_occurence,train_tokens,train_ner_tags)

In [ ]:
df.to_csv('number_ration.csv')

In [ ]:
df

In [5]:
import pandas as pd
import datasets

In [6]:
 def load_dataset_tags():

        dataset = datasets.load_dataset('nlpaueb/finer-139', split='train', streaming=True)
        dataset_tags = dataset.features['ner_tags'].feature.names
        tag2idx = {tag: int(i) for i, tag in enumerate(dataset_tags)}
        idx2tag = {idx: tag for tag, idx in tag2idx.items()}

        return tag2idx, idx2tag

In [7]:
tag2idx, idx2tag = load_dataset_tags()

In [11]:
df = pd.read_csv('number_ration.csv')

In [12]:
df

,Unnamed: 0,label,ratio
0,0,1,97.946%
1,1,2,81.941%
2,2,4,98.668%
3,3,5,99.368%
4,4,6,99.704%
...,...,...,...
135,135,164,100.0%
136,136,165,93.075%
137,137,167,99.451%
138,138,168,93.676%


In [13]:
df['real_label']=df['label'].apply(lambda x:idx2tag[x])

In [16]:
idx2tag[2]

'B-AcquiredFiniteLivedIntangibleAssetsWeightedAverageUsefulLife'

In [18]:
df.to_csv('number_ratio.csv')